In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pandas as pd 
import numpy as np 
from datasets import Dataset as TFDataset 
import pickle
from rank_bm25 import BM25Okapi
import scipy.sparse as ssp
from functools import lru_cache, partial
from tqdm import tqdm, trange
from collections import Counter, defaultdict
from transformers import PreTrainedTokenizer, AutoTokenizer

/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/bm25/cache/title_bm25.pkl', 'rb') as f:
    title_BM25 = pickle.load(f)

In [3]:
with open('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/bm25/cache/desc_bm25.pkl', 'rb') as f:
    desc_BM25 = pickle.load(f)

In [5]:
TOKENIZER_NAME = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(
        TOKENIZER_NAME,
        use_fast=False,
)

In [6]:
product_data = pd.read_csv('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv')

In [7]:
product_data['title'][102], tokenizer.encode(product_data['title'][102])

('HappyDoo - Kindergeschirr Set Mit Saugnapf - Babyteller Mit Saugnapf + Anti-Rutsch Schüssel + Löffel und Läzchen Aus Silikon - BPA Frei - Rosa - Baby Teller Mit Saugnapf - Kinderteller',
 [0,
  32506,
  8331,
  31,
  20,
  9493,
  121656,
  481,
  42,
  19943,
  3574,
  6987,
  7456,
  39557,
  20,
  24624,
  126132,
  3574,
  6987,
  7456,
  39557,
  997,
  8332,
  9,
  26951,
  66489,
  164828,
  7,
  1428,
  997,
  37300,
  54261,
  165,
  41243,
  169,
  4834,
  5489,
  215514,
  20,
  335,
  12236,
  36443,
  20,
  29641,
  20,
  24624,
  59488,
  56,
  3574,
  6987,
  7456,
  39557,
  20,
  9493,
  126132,
  2])

In [8]:
scores = title_BM25.get_scores(tokenizer.encode(product_data['title'][1000010]))
top_n = np.argsort(scores)[::-1][:10]

In [9]:
top_n

array([1355250, 1000011, 1308115, 1265529, 1192445, 1254104,  928300,
       1163272, 1352398, 1205754])

In [ ]:
product_data.iloc[1000010]['title'], product_data.iloc[1000010]['desc'], product_data.iloc[1000010]['color'], product_data.iloc[1000010]['locale']

('Amazon Brand - Eono Neck Pillow for Travel Memory Foam, Hooded Travel Pillow, Comfortable Neck Support Pillow with Hoodie for Plane Cars Trains Office, Flight Travel Cushion for Sleeping',
 'Easy to Care: The removable case makes it easy to clean when dirty, allowing it to be used for trip after trip; we recommend washing by hand; do not wash the sponge or expose to the sun; we recommend allowing the pillow to air for 3-4 hours before first use.',
 'Grey',
 'UK')

In [ ]:
product_data.iloc[1355249]['title'], product_data.iloc[1355249]['desc'], product_data.iloc[1000010]['title'], product_data.iloc[1000010]['desc']

('Amazon Brand - Eono Neck Pillow for Travel Memory Foam, Hooded Travel Pillow, Comfortable Neck Support Pillow with Hoodie for Plane Cars Trains Office, Flight Travel Cushion for Sleeping',
 'Easy to Care: The removable case makes it easy to clean when dirty, allowing it to be used for trip after trip; we recommend washing by hand; do not wash the sponge or expose to the sun; we recommend allowing the pillow to air for 3-4 hours before first use.',
 'Amazon Brand - Eono Neck Pillow for Travel Memory Foam, Hooded Travel Pillow, Comfortable Neck Support Pillow with Hoodie for Plane Cars Trains Office, Flight Travel Cushion for Sleeping',
 'Easy to Care: The removable case makes it easy to clean when dirty, allowing it to be used for trip after trip; we recommend washing by hand; do not wash the sponge or expose to the sun; we recommend allowing the pillow to air for 3-4 hours before first use.')

In [ ]:
product_data.iloc[1308114]['title'], product_data.iloc[1308114]['desc'], product_data.iloc[1265528]['title'], product_data.iloc[1265528]['desc']

('Amazon Brand - Eono Neck Pillow Memory Foam, Travel Pillow for Neck, Comfortable Neck and Head Support Pillow for Plane Cars Trains Office, Soft Flight Travel Cushion for Sleeping',
 'Easy to Care: This practical new type neck pillow case can be removed and cleaned if it is dirty, and can be used for many times. Note: wash by hand. Do not wash the sponge or expose it to the sun.',
 'Amazon Brand - Eono Neck Pillow Memory Foam, Travel Pillow for Neck, Comfortable Neck and Head Support Pillow for Plane Cars Trains Office, Soft Flight Travel Cushion for Sleeping',
 'Easy to Care: This practical new type neck pillow case can be removed and cleaned if it is dirty, and can be used for many times. Note: wash by hand. Do not wash the sponge or expose it to the sun.')

In [10]:
desc_scores = desc_BM25.get_scores(tokenizer.encode(product_data['desc'][1000010]))
desc_top_n = np.argsort(desc_scores)[::-1][:10]

In [11]:
desc_top_n

array([1000011, 1355250, 1265529, 1192445, 1308115, 1071990, 1124716,
       1279396, 1133153, 1408705])

In [12]:
product_data.iloc[1192444]['title'], product_data.iloc[1192444]['desc'], product_data.iloc[1192444]['color']

('Amazon Brand - Eono Neck Pillow Memory Foam, Travel Pillow for Neck, Comfortable Neck and Head Support Pillow for Plane Cars Trains Office, Soft Flight Travel Cushion for Sleeping',
 'Easy to Care: This practical new type neck pillow case can be removed and cleaned if it is dirty, and can be used for many times. Note: wash by hand. Do not wash the sponge or expose it to the sun.',
 'Dark Blue')

In [14]:
product_data.iloc[1133152]['title'], product_data.iloc[1133152]['desc'], product_data.iloc[1071989]['title'], product_data.iloc[1071989]['desc']

('Travel Pillow - Luchai Neck Pillow for Travel, Memory Foam Pillow Essential Travel Accessories Neck Support Pillow for Airplane,Car,Quick Pack Head Cushion for Camping,office (Blue)',
 'Easy to Clean: The zipper on the pillow case makes it easy to be detached, and it is machine washable for keeping it clean.',
 "Cute Cat Crossbody Bag Mermaid Cat Purse Gradient Little Girls Purses Kids Toddlers Cat Purse with 1 Mini Coin Wallet Purse Cat Ears Headband for Little Girl Birthday Children's Day, 3 Pieces",
 'Easy to clean: PU leather material makes theses kids purses simple to clean, no need to worry about them would get dirty after use; Please do not expose them in the sun after cleaning, just let them dry in the air naturally')

# Merge valid BM25 scores

In [3]:
product_data = pd.read_csv('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv')
title_corpus = product_data[['title']]
title_corpus = pd.concat([pd.DataFrame({'title' : ['']}), title_corpus]).reset_index(drop=True)
title_corpus['title'] = title_corpus['title'].fillna('')

In [5]:
title_corpus = TFDataset.from_pandas(title_corpus, preserve_index=False)

In [6]:
title_corpus['title']

['',
 'RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wolfram Profi Dartpfeile Set mit Flights und Schäfte',
 'Simply Keto Lower Carb* Schokodrops ohne Zuckerzusatz (750g) - Vollmilch Schoko Drops zum Naschen oder Backen - Gesüßt mit Erythrit statt Zucker - Ideal für Low-Carb & Ketogene Ernährung',
 'Sennheiser 508377 PC 5.2 Chat, Stilvolles Multi-Plattform On-Ear Headset PC, Kopfhörer mit Kabel & Unterricht, für Laptop, Telefon & PC & EPOS I Sennheiser PC 8 USB Headset, Schwarz',
 'AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbares Auto ab 1 Jahr - 4pcs Spielzeugautos Set für Mädchen',
 'PLAYMOBIL - 70522 - Cavaliere mit grauem Pony',
 'URBZUE Handwärmer, 10000mAh USB aufladbar und Digitalanzeige, elektrischer Taschenwärmer mit Mehreren Heizstufen von 40-60 ° C, tragbares warmes Wintereschenk für Frau, Mann, Kinder,rosagold',
 'Kinderkopfhörer Bluetooth, Mädchen Katzenohr Kopfhörer Over-Ear mit LED-licht Faltbare Stereo Kopfhörer Kabellose, Kopfhörer Kinder mit Mikrofon, Micro SD/TF, für H

In [7]:
with open('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/bm25/cache/title_bm25.pkl', 'rb') as f:
    title_BM25 = pickle.load(f)

In [8]:
type(title_BM25.corpus)

list